- loading data

In [22]:
! wget https://raw.githubusercontent.com/mohammedashrafdagga/ds_lr/main/dataset/melb_data.csv

--2022-01-23 16:20:52--  https://raw.githubusercontent.com/mohammedashrafdagga/ds_lr/main/dataset/melb_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2091239 (2.0M) [text/plain]
Saving to: ‘melb_data.csv.2’

melb_data.csv.2     100%[===================>]   1.99M  --.-KB/s    in 0.04s   

2022-01-23 16:20:53 (44.9 MB/s) - ‘melb_data.csv.2’ saved [2091239/2091239]



-- reading and importing libaray

In [23]:
# improt 
import pandas as pd

data_melb = pd.read_csv('melb_data.csv')

# looking a head of data
data_melb.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [24]:
data_melb.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

-- definte X and y

In [25]:
y = data_melb['Price']
X = data_melb.select_dtypes(exclude=['object'])

In [26]:
X.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,850000.0,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,1600000.0,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


In [27]:
# split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1, test_size = 0.2)

In [28]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10864, 13)
(2716, 13)
(10864,)
(2716,)


-- How Know if have missing value or not

In [29]:
# isnull()
X.isnull().any()

Rooms            False
Price            False
Distance         False
Postcode         False
Bedroom2         False
Bathroom         False
Car               True
Landsize         False
BuildingArea      True
YearBuilt         True
Lattitude        False
Longtitude       False
Propertycount    False
dtype: bool

In [30]:
# return just columns with missing value
col_with_missing_value = [col for col in X.columns if X[col].isnull().sum() > 0]
col_with_missing_value

['Car', 'BuildingArea', 'YearBuilt']

## Approaches working with Missing Value

#### 1. drop columns containing missing value

In [31]:
# is better make copy of data
X_train_missing = X_train.copy()
X_test_missing = X_test.copy()

before drop columns containing missing value

In [32]:
print(X_train_missing.shape)
print(X_test_missing.shape)

(10864, 13)
(2716, 13)


In [33]:
X_train_missing = X_train_missing.drop(col_with_missing_value, axis = 1)
X_test_missing = X_test_missing.drop(col_with_missing_value, axis = 1)

After drop columns containing missing value

In [34]:
print(X_train_missing.shape)
print(X_test_missing.shape)

(10864, 10)
(2716, 10)


In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Calaluate MAE after apply Approaches 1

In [37]:
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(X_train_missing, X_test_missing, y_train, y_test))

MAE from Approach 1 (Drop columns with missing values):
349.3657216494846


###  Approach 2 (Imputation)

In [38]:
from sklearn.impute import SimpleImputer

# make object from smapleImputer 
# defalut stragey is calcalute mean and set
imputer = SimpleImputer()

In [40]:


# apply imputeation on training and testing data

X_train_imputer = pd.DataFrame(imputer.fit_transform(X_train))
X_test_imputer = pd.DataFrame(imputer.transform(X_test))

In [41]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(X_train_imputer, X_test_imputer, y_train, y_test))

MAE from Approach 2 (Imputation):
379.212407952872


### Approach 3 (An Extension to Imputation)

In [43]:
# make a copy of data beacuse add columns 
X_train_plus = X_train.copy()
X_test_plus = X_test.copy()

In [46]:
# add columns news refreance update value
for col in col_with_missing_value:
  X_train_plus[col + "_with_missing"] = X_train_plus[col].isnull()
  X_test_plus[col + "_with_missing"] = X_test_plus[col].isnull()

In [47]:
impute = SimpleImputer()


X_train_impute = pd.DataFrame(impute.fit_transform(X_train_plus))
X_test_impute = pd.DataFrame(impute.transform(X_test_plus))

In [49]:
# update columns name
X_train_impute.columns = X_train_plus.columns
X_test_impute.columns = X_test_plus.columns

In [50]:
X_train_impute.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_with_missing,BuildingArea_with_missing,YearBuilt_with_missing
0,3.0,1635000.0,11.2,3186.0,3.0,1.0,2.0,366.0,156.000000,1920.000000,-37.90380,145.00010,10579.0,0.0,0.0,0.0
1,3.0,725000.0,7.8,3058.0,3.0,1.0,0.0,238.0,131.000000,1900.000000,-37.75390,144.96120,11204.0,0.0,0.0,0.0
2,3.0,1280000.0,5.2,3056.0,3.0,1.0,1.0,439.0,153.363129,1964.948871,-37.77047,144.97005,11918.0,0.0,1.0,1.0
3,2.0,772000.0,11.4,3163.0,2.0,1.0,2.0,0.0,100.000000,1973.000000,-37.88630,145.06600,7822.0,0.0,0.0,0.0
4,4.0,1360000.0,11.0,3018.0,4.0,2.0,4.0,615.0,153.363129,1964.948871,-37.87057,144.83623,5301.0,0.0,1.0,1.0


In [51]:
print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(X_train_impute, X_test_impute, y_train, y_test))

MAE from Approach 3 (An Extension to Imputation):
372.0206553755524


## Best One is First one beacuse getting Min Mean Absolute Error